## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

In [ ]:
import numpy as np
import pandas as pd
from numba import vectorize
from numba import guvectorize
from numba import jit, njit
import re

In [ ]:
%%time
def sa(a, b):
  return np.sum(a * b)
a1 = np.arange(1, 13).reshape(3, 4)
print(a1)
#sa = np.vectorize(sa, signature = '(i),(i)->()')
print(sa(a1, np.ones(4)))

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]
78.0
CPU times: user 1.26 ms, sys: 37 µs, total: 1.3 ms
Wall time: 5.05 ms


## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [ ]:
#%%timeit
A = np.random.randint(0, 1000, 1000000)
B = np.asarray([i + 100 for i in A])
c = np.mean(B)
print(c)

598.860906


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [ ]:
%%time
@njit
def gen_val(sup):
  k = []
  for i in range(2000000):
    l = np.random.choice(25, 5)
    k1 = []
    for y in l:
      k1.append(sup[y])
    k.append(''.join(k1))
  return k
sup = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'G', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'Q', 'W', 'V', 'Y', 'X', 'Z', 'J', 'U']
table = pd.DataFrame(np.random.randint(1, 100, size = [2000000, 4]))
key_val = gen_val(sup)
table['key'] = key_val
table

/usr/local/lib/python3.10/dist-packages/numba/core/ir_utils.py:2147: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'sup' of function 'gen_val'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<timed exec>", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


CPU times: user 2.63 s, sys: 186 ms, total: 2.81 s
Wall time: 2.82 s


,0,1,2,3,key
0,38,42,91,95,HICWW
1,33,68,68,88,XGKVA
2,14,7,20,88,GNXAE
3,68,15,37,88,HXYDO
4,41,42,45,95,YSZQL
...,...,...,...,...,...
1999995,4,41,95,11,GKHHM
1999996,69,57,91,5,KEZWI
1999997,86,37,75,68,SOAIE
1999998,16,94,71,79,ZATTT


In [ ]:
%%time
def correct(table):
  df_filtered = table[table['key'].apply(lambda x: bool(re.match('^[A-E]{5}$', x)))]
  return df_filtered
correct(table)

CPU times: user 1.5 s, sys: 650 µs, total: 1.5 s
Wall time: 1.51 s


,0,1,2,3,key
1376,82,7,45,50,DEBBB
1434,3,48,30,52,DCACC
2307,82,89,60,68,BEDCB
8319,92,26,22,52,CABEC
9416,51,25,25,69,ECEEE
...,...,...,...,...,...
1980431,22,43,34,27,AEECD
1982119,35,54,77,59,ADEAC
1982611,22,97,95,2,BDCDD
1988623,61,19,39,73,BAACD


## Лабораторная работа 3

In [ ]:
!pip install line_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [ ]:
with open('recipes_sample.csv') as file:
  recipes = pd.read_csv(file)
with open('reviews_sample.csv') as file:
  reviews = pd.read_csv(file)
  reviews['date'] =  pd.to_datetime(reviews['date'])

FileNotFoundError: ignored

In [ ]:
%%time
def a_mean(table):
  iter = table.iterrows()
  new_table = []
  for i in range(len(table)):
    el = next(iter)
    if el[1][3].year == 2010:
      new_table.append(el[1][4])
  return sum(new_table)/len(new_table)
a_mean(reviews)

CPU times: user 5.17 s, sys: 10.5 ms, total: 5.18 s
Wall time: 6.06 s


4.4544402182900615

In [ ]:
%%time
def b_mean(table):
  means = []
  iter = table.iterrows()
  for i in range(len(table)):
    el = next(iter)
    means.append(el[1][4])
  return sum(means)/len(means)
reviews1 = reviews[reviews['date'].dt.year == 2010]
b_mean(reviews1)

CPU times: user 456 ms, sys: 195 µs, total: 456 ms
Wall time: 456 ms


4.4544402182900615

In [ ]:
%%time
def c_mean(table):
  k = table['rating']
  return(k.mean(0))
c_mean(reviews1)

CPU times: user 587 µs, sys: 0 ns, total: 587 µs
Wall time: 597 µs


4.4544402182900615

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

Timer unit: 1e-09 s

Total time: 10.6096 s
File: /content/mprun_demos.py
Function: a_mean at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def a_mean(table):
     2         1       5166.0   5166.0      0.0    iter = table.iterrows()
     3         1        324.0    324.0      0.0    new_table = []
     4    126696   44145823.0    348.4      0.4    for i in range(len(table)):
     5    126696 9601187553.0  75781.3     90.5      el = next(iter)
     6    114602  894291357.0   7803.5      8.4      if el[1][3].year == 2010:
     7     12094   69918915.0   5781.3      0.7        new_table.append(el[1][4])
     8         1      96801.0  96801.0      0.0    return sum(new_table)/len(new_table)

In [ ]:
%%time
def b_mean_ed(table):
  means = [table.iloc[i, 4] for i in range(len(table))]
  return sum(means)/len(means)
reviews1 = reviews[reviews['date'].dt.year == 2010]
b_mean_ed(reviews1)

NameError: ignored

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [ ]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [ ]:
%%time
get_word_reviews_count(reviews)

CPU times: user 20.6 s, sys: 55.6 ms, total: 20.7 s
Wall time: 20.7 s


{'Last': 94,
 'week': 804,
 'whole': 5628,
 'sides': 312,
 'of': 109029,
 'frozen': 2647,
 'salmon': 729,
 'fillet': 60,
 'was': 88781,
 'on': 34583,
 'sale': 149,
 'in': 61539,
 'my': 44144,
 'local': 561,
 'supermarket,': 10,
 'so': 46090,
 'I': 285147,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13965,
 '3,': 48,
 'but': 42513,
 'total': 381,
 'weight': 160,
 'over': 9065,
 '10': 2303,
 'pounds).': 2,
 '': 214145,
 'This': 39448,
 'recipe': 41098,
 'is': 55075,
 'perfect': 4398,
 'for': 121224,
 'fillet,': 14,
 'even': 7878,
 'though': 2314,
 'it': 111175,
 'calls': 520,
 'steaks.': 93,
 'cut': 6688,
 'up': 13585,
 'the': 266050,
 'into': 7031,
 'individual': 314,
 'portions': 156,
 'and': 217849,
 'followed': 4859,
 'instructions': 731,
 'exactly.': 571,
 "I'm": 7145,
 'one': 15086,
 'those': 2287,
 'food': 2413,
 'combining': 74,
 'diets,': 5,
 'left': 4690,
 'out': 23644,
 'white': 3425,
 'wine': 1256,
 'added': 21710,
 'just': 24944,
 'a': 166136,
 'dash': 532,
 'vineg

Уязвимые места (по профайлеру):
- {method 'append' of 'list' objects} (много элементов)
- {method 'split' of 'str' objects} (много отзывов - много создания списков)
-  Для каждого слова проверяется наличие в словаре word_reviews и, если слова там нет, создается пустой список. Это может быть затратно при большом количестве уникальных слов, так как создается много пустых списков
-  В цикле по отзывам дважды проходится по каждому слову: первый раз для создания списка word_reviews, второй раз для подсчета количества отзывов с этим словом.


In [ ]:
from collections import defaultdict
def get_word_reviews_count1(df):
    word_reviews = defaultdict(list)
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = [word for word in review.split(' ')]
        for word in words:
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for word, reviews in word_reviews.items():
        word_reviews_count[word] = len(reviews)
    return word_reviews_count

In [ ]:
%%time
get_word_reviews_count1(reviews)

CPU times: user 11.3 s, sys: 37.9 ms, total: 11.3 s
Wall time: 11.4 s


{'Last': 94,
 'week': 804,
 'whole': 5628,
 'sides': 312,
 'of': 109029,
 'frozen': 2647,
 'salmon': 729,
 'fillet': 60,
 'was': 88781,
 'on': 34583,
 'sale': 149,
 'in': 61539,
 'my': 44144,
 'local': 561,
 'supermarket,': 10,
 'so': 46090,
 'I': 285147,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13965,
 '3,': 48,
 'but': 42513,
 'total': 381,
 'weight': 160,
 'over': 9065,
 '10': 2303,
 'pounds).': 2,
 '': 214145,
 'This': 39448,
 'recipe': 41098,
 'is': 55075,
 'perfect': 4398,
 'for': 121224,
 'fillet,': 14,
 'even': 7878,
 'though': 2314,
 'it': 111175,
 'calls': 520,
 'steaks.': 93,
 'cut': 6688,
 'up': 13585,
 'the': 266050,
 'into': 7031,
 'individual': 314,
 'portions': 156,
 'and': 217849,
 'followed': 4859,
 'instructions': 731,
 'exactly.': 571,
 "I'm": 7145,
 'one': 15086,
 'those': 2287,
 'food': 2413,
 'combining': 74,
 'diets,': 5,
 'left': 4690,
 'out': 23644,
 'white': 3425,
 'wine': 1256,
 'added': 21710,
 'just': 24944,
 'a': 166136,
 'dash': 532,
 'vineg

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


#### [версия 2]
* Уточнены формулировки задач 1, 3, 4